<a href="https://colab.research.google.com/github/sadnow/RIFE-Google-Colab-Simplified/blob/main/motion_smoothing_VQGAN%2BCLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplified RIFE Motion Smoothing (Trimmed for this speific use by sadnow)

For whatever reason this one makes the best motion smoothing with minimal messing around with setings.

Original script based on "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. Trimmed most of it out becuase I don't use the other parts.

This script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video.

In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

# Mount Google Drive (Optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Motion interpolation (RIFE Smoothing)

"(...) (O)utput can be a bit wobbly, so video interpolation can be used to smooth things out a bit."


In [ ]:
print("Installing libraries for motion smoothing.")
!git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
!gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
!7z e RIFE_trained_model_HDv2.zip
!mkdir /content/RIFE/train_log
!mv *.pkl /content/RIFE/train_log/
%cd /content/RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt
%cd /content/
print("Done.")
%cd /content/RIFE/
print("Installing libraries for motion smoothing.")
# @title Settings
input_path='/content/drive/MyDrive/VQLIPSE/videos/rickmorty.mp4' #@param {type:"string"}
target_fps=60#@param {type:"raw"}
length_multiplier=1#@param {type:"integer"}
#@markdown
#@markdown `length_multiplier` can be set to 1 if you don't want to increase the length. Set it to 2 if you want to double your video length, 4 to quadruple, etc.
#@markdown
#@markdown ---
#@markdown Huge tip: If your length_multiplier is =>2 and you re-run the same script (without changing settings), it will automatically double/square(?) your already-doubled output from before!

!python3 /content/RIFE/inference_video.py --fps=$target_fps --exp=$length_multiplier --video=$input_path --skip
%cd /content/

# How to View the Output

Go to your Google Drive and you will find the output in the same folder as your input_path